<a href="https://colab.research.google.com/github/amilyk/Agentic-ai-notes/blob/main/SQL_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip uninstall docstring-parser
!pip install --upgrade "docstring-parser>=0.15"
!python -m pip install aisuite --ignore-installed docstring-parser

Found existing installation: docstring_parser 0.17.0
Uninstalling docstring_parser-0.17.0:
  Would remove:
    /usr/local/lib/python3.12/dist-packages/docstring_parser-0.17.0.dist-info/*
    /usr/local/lib/python3.12/dist-packages/docstring_parser/*
Proceed (Y/n)? n
  Using cached aisuite-0.1.13-py3-none-any.whl.metadata (13 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached docstring_parser-0.14.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
Using cached aisuite-0.1.13-py3-none-any.whl (84 kB)
Using cached docstring_parser-0.14.1-py3-none-any.whl (33 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 

In [1]:
# !touch .env
# !echo "DEEPSEEK_API_KEY='your-deepseek-api'" >> .env

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [4]:
import aisuite as ai

client = ai.Client()

In [5]:
import sql_utils
sql_utils.create_transactions_db()

SQLite database 'products.db' created with a single 'transactions' table (event-sourced).


In [6]:
sql_utils.print_html(sql_utils.get_schema('products.db'))

# query SQL通过LLM

In [7]:
def generate_sql(question: str, schema: str, model: str) -> str:
    prompt = f"""
    You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

    Schema:
    {schema}

    User question:
    {question}

    Respond with the SQL only.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content.strip()

In [8]:
# Example usage of generate_sql

# We provide the schema as a string
schema = """
Table name: transactions
id (INTEGER)
product_id (INTEGER)
product_name (TEXT)
brand (TEXT)
category (TEXT)
color (TEXT)
action (TEXT)
qty_delta (INTEGER)
unit_price (REAL)
notes (TEXT)
ts (DATETIME)
"""

# We ask a question about the data in natural language
question = "Which color of product has the highest total sales?"

sql_utils.print_html(question, title="User Question")

# Generate the SQL query using the specified model
sql_V1 = generate_sql(question, schema, model="deepseek:deepseek-coder")

# Display the generated SQL query
sql_utils.print_html(sql_V1, title="SQL Query V1")

In [11]:
sql_V1

"```sql\nSELECT color, SUM(qty_delta * unit_price) AS total_sales\nFROM transactions\nWHERE action = 'sale' AND qty_delta > 0\nGROUP BY color\nORDER BY total_sales DESC\nLIMIT 1;\n```"

发现qty_delta里都是负的，查询里where条件加上qty_delta > 0，就会导致查询为空

In [31]:
test_sql = 'SELECT color,qty_delta,unit_price AS total_sales\nFROM transactions\nWHERE action = "sale"\nORDER BY qty_delta DESC\nLIMIT 10;'
sql_utils.execute_sql(test_sql, db_path='products.db')

,color,qty_delta,total_sales
0,red,-1,51.84
1,red,-1,51.84
2,red,-1,56.05
3,red,-1,57.16
4,green,-1,27.95
5,green,-1,25.94
6,green,-1,30.75
7,green,-1,30.75
8,black,-1,20.82
9,green,-1,26.13


,color,total_sales
0,blue,-190571.46


,color,total_sales
0,white,358315.09


In [9]:
#Query Execution
df_sql_V1 = sql_utils.execute_sql(sql_V1, db_path='products.db')

#sql一样，但执行缺没结果
sql_utils.print_html(df_sql_V1, title="Output of SQL Query V1 - ❌ Does NOT fully answer the question")

color,total_sales


# 用反思提升sql查询能力

In [51]:
def refine_sql(
    question: str,
    sql_query: str,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    Reflect on whether a query's *shown output* answers the question,
    and propose an improved SQL if needed.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL OUTPUT fully answers the user's question.
Step 2: If improvement is needed, provide a refined SQL query for SQLite.
If the original SQL is already correct, return it unchanged.

Return STRICT JSON with two fields:
{{
  "feedback": "<1-3 sentences explaining the gap or confirming correctness>",
  "refined_sql": "<final SQL to run>"
}}
"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if model doesn't return valid JSON
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql

In [54]:
# Example: refine the generated SQL (V1 → V2)

feedback, sql_V2 = refine_sql(
    question=question,
    sql_query=sql_V1,   # <- comes from generate_sql() (V1)
    schema=schema, # <- we reuse the schema from section 3.1
    model="deepseek:deepseek-coder"
)

# Display the original prompt
sql_utils.print_html(question, title="User Question")

# --- V1 ---
sql_utils.print_html(sql_V1, title="Generated SQL Query (V1)")

# Execute and show V1 output
df_sql_V1 = sql_utils.execute_sql(sql_V1, db_path='products.db')
sql_utils.print_html(df_sql_V1, title="SQL Output of V1 - ❌ Does NOT fully answer the question")

# --- Feedback + V2 ---
sql_utils.print_html(feedback, title="Feedback on V1")
sql_utils.print_html(sql_V2, title="Refined SQL Query (V2)")

# Execute and show V2 output
df_sql_V2 = sql_utils.execute_sql(sql_V2, db_path='products.db')
sql_utils.print_html(df_sql_V2, title="SQL Output of V2 - ❌ Does NOT fully answer the question")

color,total_sales


color,total_sales


# 增加外部执行器（把V1 sql执行结构作为反思的参考）

In [63]:
import pandas as pd
import json

def refine_sql_external_feedback(
    question: str,
    sql_query: str,
    df_feedback: pd.DataFrame,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    Evaluate whether the SQL result answers the user's question and,
    if necessary, propose a refined version of the query.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
    You are a SQL reviewer and refiner.

    User asked:
    {question}

    Original SQL:
    {sql_query}

    SQL Output:
    {df_feedback.to_markdown(index=False)}

    Table Schema:
    {schema}

    Step 1: Briefly evaluate if the SQL output answers the user's question.
    Step 2: If the SQL could be improved, provide a refined SQL query.
    If the original SQL is already correct, return it unchanged.

    Return a strict JSON object with two fields:
    - "feedback": brief evaluation and suggestions
    - "refined_sql": the final SQL to run
    """

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )


    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if the model does not return valid JSON:
        # use the raw content as feedback and keep the original SQL
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql

In [59]:
#  Example: Refine SQL with External Feedback (V1 → V2)

# Execute the original SQL (V1)
df_sql_V1 = sql_utils.execute_sql(sql_V1, db_path='products.db')

# Use external feedback to evaluate and refine
feedback, sql_V2 = refine_sql_external_feedback(
    question=question,
    sql_query=sql_V1,   # V1 query
    df_feedback=df_sql_V1,    # Output of V1
    schema=schema,
    model="deepseek:deepseek-coder"
)

# --- V1 ---
sql_utils.print_html(question, title="User Question")
sql_utils.print_html(sql_V1, title="Generated SQL Query (V1)")
sql_utils.print_html(df_sql_V1, title="SQL Output of V1 - ❌ Does NOT fully answer the question")

# --- Feedback & V2 ---
sql_utils.print_html(feedback, title="Feedback on V1")
sql_utils.print_html(sql_V2, title="Refined SQL Query (V2)")

# Execute and display V2 results
df_sql_V2 = sql_utils.execute_sql(sql_V2, db_path='products.db')
sql_utils.print_html(df_sql_V2, title="SQL Output of V2 (with External Feedback) - ✅ Fully answers the question")

color,total_sales


color,total_sales


# 数据库查询工作流（包含反思）

In [60]:
def run_sql_workflow(
    db_path: str,
    question: str,
    model_generation: str = "openai:gpt-4.1",
    model_evaluation: str = "openai:gpt-4.1",
):
    """
    End-to-end workflow to generate, execute, evaluate, and refine SQL queries.

    Steps:
      1) Extract database schema
      2) Generate SQL (V1)
      3) Execute V1 → show output
      4) Reflect on V1 with execution feedback → propose refined SQL (V2)
      5) Execute V2 → show final answer
    """

    # 1) Schema
    schema = sql_utils.get_schema(db_path)
    sql_utils.print_html(
        schema,
        title="📘 Step 1 — Extract Database Schema"
    )

    # 2) Generate SQL (V1)
    sql_v1 = generate_sql(question, schema, model_generation)
    sql_utils.print_html(
        sql_v1,
        title="🧠 Step 2 — Generate SQL (V1)"
    )

    # 3) Execute V1
    df_v1 = sql_utils.execute_sql(sql_v1, db_path)
    sql_utils.print_html(
        df_v1,
        title="🧪 Step 3 — Execute V1 (SQL Output)"
    )

    # 4) Reflect on V1 with execution feedback → refine to V2
    feedback, sql_v2 = refine_sql_external_feedback(
        question=question,
        sql_query=sql_v1,
        df_feedback=df_v1,          # external feedback: real output of V1
        schema=schema,
        model=model_evaluation,
    )
    sql_utils.print_html(
        feedback,
        title="🧭 Step 4 — Reflect on V1 (Feedback)"
    )
    sql_utils.print_html(
        sql_v2,
        title="🔁 Step 4 — Refined SQL (V2)"
    )

    # 5) Execute V2
    df_v2 = sql_utils.execute_sql(sql_v2, db_path)
    sql_utils.print_html(
        df_v2,
        title="✅ Step 5 — Execute V2 (Final Answer)"
    )

In [62]:
run_sql_workflow(
    "products.db",
    "Which color of product has the highest total sales?",
    model_generation="deepseek:deepseek-chat",
    model_evaluation="deepseek:deepseek-chat"
)

color,total_sales


color,total_sales


# Deepseek sql query返回为空的原因

1.   通过查询数据库发现若为sale，qty_delta都为负值
2.   而deepseek生成的sql中都增加了条件qty_delta>0，因此2次查询都返回空

In [ ]:
test_sql = 'SELECT color, SUM(qty_delta * unit_price) AS total_sales\nFROM transactions\nWHERE action = "sale"\nGROUP BY color\nORDER BY total_sales DESC\nLIMIT 1;'
sql_utils.execute_sql(test_sql, db_path='products.db')

# 对比课程中gpt4.1生成的V1 sql和V2 sql（带外部结果的反思）

In [34]:
gpt4_1_V1_sql = 'SELECT color, SUM(qty_delta * unit_price) AS total_sales\nFROM transactions\nWHERE action = "sale"\nGROUP BY color\nORDER BY total_sales DESC\nLIMIT 1;'
sql_utils.execute_sql(gpt4_1_V1_sql, db_path='products.db')

,color,total_sales
0,blue,-190571.46


In [35]:
gpt4_1_V2_sql = 'SELECT color, SUM(-qty_delta * unit_price) AS total_sales\nFROM transactions\nWHERE action = "sale"\nGROUP BY color\nORDER BY total_sales DESC\nLIMIT 1;'
sql_utils.execute_sql(gpt4_1_V2_sql, db_path='products.db')

,color,total_sales
0,white,358315.09
